# **Saving Ashman & Menken: How to Train up an AI Disney Lyracist, long passed**

![alt text](https://d23.com/app/uploads/2016/09/780w-463h_092616_oral-history-howard-ashman-1.jpg)

##What are you using Machine Learning for? 
---------------------------------------------
Like so many celebrated Disney Lyracists, Howard was extremely overworked and  woefully underpaid!(Kidding!) He passed away in the 1990's but his long-time partner Alan Menken still misses him. Together, they were resposible for some pretty iconic Disney soundtracks. What if Alan could have a version of his old partner back again?  </br>
Together they wrote Little Mermaid, Aladdin and Beauty and the Beast. Menken went on to write Pocahontas, Newsies, The Hunchback of Notre Dame, Hercules, Home on the Range, Enchanted, and Tangled, among others.
Alan is hoping that AI will help him knock out some new hit Disney songs with Howard's old lyrics flair. 


## **What we are aiming for:** 
**Metrics:** Menken once said that his songs should be hummable, meaning catchy, and Ashman was a playwright that also thought that they should sound natural, so that's what we are aiming for!  
</br> 
Our resuslts will pit two Disney songs against one another and you will choose which was written by Ashman and which was not so that Alan can decide if he'd like his old partner back as an AI.  

</br>

## **Here Are a couple of real songs from Ashman and Menken** 

### **Part of Your World**
<p>
Look at this stuff, isnt it neat? </br> 
Wouldn't you think my collection's complete? </br>
Wouldn't you think Im the girl,
the girl who has everything?
</p>


!["Part of Your World"](https://media.giphy.com/media/ok1tOvpe5yvy8/giphy.gif)


</br>

### **A Whole New World**
<p>
I can show you the world: shining, shimmering, splendid!  </br> 
Tell me Princess, now when did you last let your heart decide? 
</p>

!["A Whole New World"](https://media.giphy.com/media/jyJKxPKMTy36w/giphy.gif)

</br>




### **(Not Ashman and Menken, but due to the dataset we get the old family classic) Lester's Possum Park**
<p>
Now gather 'round my possum pals, join the jamboree! </br>
Come hoot and howl and holler from the heart</br>
Every chicken, pig ,and goat will help </br>
by yelpin' out a yodel here at <br>
Lester's Possum Park!

</p>

!["Lester's Possum Park"](https://media.giphy.com/media/n1NKE2VJ0h6MM/giphy.gif)





 


## **What we got:**  
Givin the current state of AI unupervised text generation models, and the  dataset for used for pretraining (us),  What Alan Menken is actually in store for is probably interesting but not quite what he's looking for.

**My Burrito's Whole Funky Sloth Park (anyone?)**
<p>
Just Imagine what these lyrics might say... I assure you it was worse. 

</p>

!["Burrito's New Sloth Park"](https://media.giphy.com/media/x4gaYH1e2OsVO/giphy.gif) 

</br>

But then it got better, and we will explore results at the end! 

# **Data**

The first thing we need are some Disney song lyrics! Unfortunately for both Alan and I, Disney owns the rights to those lyrics, so we had to rely on scraped lyrics from [All the Lyrics ](https://allthelyrics.com), which can be tricky as people input these lyrics. 
The first step was to write a scraper that used Beautiful Soup to grab the lyrics of every song on the site tagged "Disney" and write the results to a .csv file, "songs.csv".  

 

In [0]:
#!/usr/bin/env python
# Imports 
from bs4 import BeautifulSoup
from langdetect import detect
import urllib.request
import collections
import csv
import logging
import re
import sys

BASE_URL = 'https://www.allthelyrics.com'
FILTER_ELS = '/lyrics/disney'
LANGS = ('en', )
CSV_OUT = 'songs.csv'
SEP = ','

# Python namedtuple
Song = collections.namedtuple('Song', 'album title lyrics')

logging.basicConfig(level=logging.DEBUG)


def validate_en(lyrics):
    """Validate that song lyrics are written in English

    Arguments:
        lyrics (str): Song lyrics

    Returns:
        bool: Whether the lyrics are English or not
    """

    l = detect(lyrics)
    logging.info('Language: {}'.format(l))
    if l in LANGS:
        return True
    else:
        return False


def get_lyrics_data(url):
    """Fetch and extract lyrics data from an all the lyrics URL

    Arguments:
        url (str): an AllTheLyrics song URL

    Returns: 
        Song: A named tuple containing song lyric data
    """
    source = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(source, 'html.parser')
    title, album, lyrics = None, None, None

    # song title
    # get page title first...
    title = soup.title.string
    # then sanitize page title into song title
    title = re.sub('^Disney – ', '', title)
    title = re.sub(' lyrics', '', title)
    logging.info('Title: {}'.format(title))

    # album
    for a in soup.find_all('div', class_='content-text-album'):
        album = a.text
        # strip leading string from album div text
        album = re.sub('Album: ', '', album)
    logging.info('Album {}'.format(album))

    # lyrics
    for d in soup.find_all('div', class_='content-text-inner'):
        lyrics_raw = d.text
        if validate_en(lyrics_raw):
            lyrics = lyrics_raw
            # Some songs lyrics are written(character)
            #Lyric which will confuse the system. Add a space to fix this.
            lyrics = re.sub('\)', ') ', lyrics)
            lyrics = re.sub('[ ]{1,}', ' ', lyrics)
            # Some songs have character :
            # lyric -OR- (character) : lyric. 
            # Drop leding space from colon character
            lyrics = re.sub(' :', ':', lyrics)
            # Some songs have [character:] lyric. 
            # Drop the internal colon character.
            lyrics = re.sub(':] ', '] ', lyrics)
            return Song(album, title, lyrics)
        else:
            raise ValueError('Non-English song')


def fetch_urls_from_index(url):
    """Fetch song links from an AllTheLyrics index URL
    
    Arguments:
        url (str): an AllTheLyrics index URL

    Returns: 
        list: A list of AllTheLyrics song URLs
    """
    urls = []
    source = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(source, 'html.parser')
    for t in soup.find_all('a'):
        href = t.get('href')
        # all lyrics pages start with /lyrics
        if href.startswith('/lyrics'):
            urls.append(BASE_URL + href)

    logging.info('Candidate URLs: {}'.format(len(urls)))
    return urls


def main():
    song_count = 0
    with open(CSV_OUT, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=SEP, quoting=csv.QUOTE_MINIMAL)
        for u in fetch_urls_from_index(BASE_URL + FILTER_ELS):
            try:
                song = get_lyrics_data(u)
                writer.writerow(list(song))
                song_count = song_count + 1
            except Exception:
                logging.exception('Skipped')

    logging.info('Songs extracted: {}'.format(song_count))


if __name__ == '__main__':
    sys.exit(main())

### Cleaning
I had to do a bit of hand cleaning and curation on the data, which were 417 complete sets of lyrics in order to: 
* Ensure the songs actually were Disney songs. (nearly all were, but some VERY obscure) 
* Fill in as much of the missing information as possible:  Some songs were missing titles, some had incorrect titles and many were missing the source of  the song.  
 

# **So what do we need to do to train this data?**

![Straight Shooter](https://media.giphy.com/media/F0DHF9GOM0DzG/giphy.gif)

We need to give the  data to the model straight, so we don't confuse it...too much. 

### Transforming and Formatting 
The Model, as we wil see later, needs the songs needed to be added as a .txt file so it will have an idea of what we want back. We need to give it an explicit template:  </br>

TITLE: Insert title here </br>

Lyrics</br>
Lyrics</br>
Lyrics</br>
<p>--- </br>
The following formatter accomplished this task nicely!  

In [0]:
"""
Formatter

Transforms the songs.csv data set into a single text file
suitable for fine-tuning a GPT2 model. The format is as follows:

SOURCE - Source here 
TITLE: Insert title here

Lyrics
Lyrics
Lyrics
---

"""
#!/usr/bin/env python

import csv
import sys

CSV_IN = 'songs.csv'
SEP = ','
TEXT_OUT = 'songs.txt'

def main():
  with open(TEXT_OUT, 'w') as text_file:
      with open(CSV_IN, 'r', newline='') as csvfile:
          reader = csv.reader(csvfile, delimiter=SEP, quoting=csv.QUOTE_MINIMAL)
          for row in reader:
              text_file.write('SOURCE - {}\n\n'.format(row[0]))
              text_file.write('TITLE: {}\n\n'.format(row[1]))
              text_file.write('{}---\n\n'.format(row[2]))


if __name__ == '__main__':
    sys.exit(main())


## Now we have a nice shiny text file, ready to be used for training our model! Here's an example:  

In [0]:

TITLE: A Whole New World

(Aladdin) I can show you the world
Shining, shimmering, splendid
Tell me, princess, now when did
You last let your heart decide?
I can open your eyes
Take you wonder by wonder
Over, sideways and under
On a magic carpet ride
A whole new world
A new fantastic point of view
No one to tell us no
Or where to go
Or say we're only dreaming
(Jasmine) A whole new world
A dazzling place I never knew
But when I'm way up here
It's crystal clear
that now I'm in a whole new world with you
(Aladdin) Now I'm in a whole new world with you
(Jasmine) Unbelievable sights
Indescribable feeling
Soaring, tumbling, freewheeling
Through an endless diamond sky
(Jasmine) A whole new world
(Aladdin) Don't you dare close your eyes
(Jasmine) A hundred thousand things to see
(Aladdin) Hold your breath - it gets better
(jasmine) I'm like a shooting star
I've come so far
I can't go back to where I used to be
(Aladdin) A whole new world
(Jasmine) Every turn a surprise
(Aladdin) With new horizons to pursue
(Jasmine) Every moment gets better
(Both) I'll chase them anywhere
There's time to spare
Let me share this whole new world with you
(Aladdin) A whole new world
(Jasmine) A whole new world
(Aladdin) That's where we'll be
(Jasmine) That's where we'll be
(Aladdin) A thrilling chase
(Jasmine) A wondrous place
(Both) For you and me
---

...

![What Do You Need?](https://media.giphy.com/media/t7NqkPey14axG/giphy.gif)

#**Training The Model**

## Model : I used a Language Model called a Transformer.
![Transitron from Toy Story of Terror!](https://vignette.wikia.nocookie.net/pixar/images/0/03/TransitronToyStoryOfTerror1.png/revision/latest?cb=20131019015524)</br>
(I desperately wanted Transformers to be owned by Disney.  They are not. However, Transitron does exist in the [Pixar Toy Story Universe](https://pixar.fandom.com/wiki/Transitron), and is therefore fair game! )
</br>
(**Not That one though!  THIS one:**)</br>
![Transformer](https://miro.medium.com/max/2254/1*Aqcm4iX3AQNWx9Zb-z7o1Q.png)

## A Language Model is basically a machine learning model that is able to look at part of a sentence and predict the next word.</br>
The most famous language models are smartphone keyboards that suggest the next word based on what you’ve currently typed.

**The Transformer** is a [model](https://towardsdatascience.com/transformers-141e32e69591) that uses attention to boost the speed with which Language models can be trained.

GPT-2 (a successor to GPT), is one of several Lanuage models. 
* large transformer-based language model with 1.5 billion parameters
* trained on a dataset of over 8 million web pages 
* trained simply to predict the next word in 40GB of Internet text.</br> 
* simple objective: predict the next word, given all of the previous words within some text. </br>

## [From the GPT-2 Team- ](https://openai.com/blog/better-language-models/)
> We created a new dataset which emphasizes diversity of content, by scraping content from the Internet. In order to preserve document quality, we used only pages which have been curated/filtered by humans—specifically, we used outbound links from Reddit which received at least 3 karma. This can be thought of as a heuristic indicator for whether other users found the link interesting (whether educational or funny), leading to higher data quality than other similar datasets, such as CommonCrawl.</br>

(**They used Reddit! What could possibly go wrong?**)

> GPT-2 generates synthetic text samples in response to the model being primed with an arbitrary input. The model is chameleon-like—it adapts to the style and content of the conditioning text. This allows the user to generate realistic and coherent continuations about a topic of their choosing


## [The Illustrated GPT-2 (Visualizing Transformer Language Models)](http://jalammar.github.io/illustrated-gpt2/)


## **How do I get my hands on this?**

I used a Library called [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple) by Max Woolf and spun out my own Colab notebook to do my training. 

* spin up a single google colaboratory notebook
* load various sizes of the GPT-2 model (I used 355 MB) 
* fine-tune using TensorFlow 
* complimentary GPU provided by google colaboratory
* generate synthetic text passages 
 







## How to Train a GPT-2 Text-Generating Model w/ GPU 

Retrain an advanced text generating neural network on any text dataset on a GPU using Collaboratory using `gpt-2-simple` library.

For more about `gpt-2-simple`, visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). See here: [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information. 

* Import tensorflow V1.x (gpt-2-simple only uses 1 )
* Install the library gpt-2-simple
* import datetime
* import needed files 


In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

## About the GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing me to train the larger GPT-2 models and generate more text.

Verify which GPU is active after running the cell.  

In [0]:
!nvidia-smi

## Downloading GPT-2

Since I am retraining a model on new text (the lyrics), I need to download the GPT-2 model first. 

There are three released sizes of GPT-2:</br>
#### Small

* `124M` (default): the "small" model, 500MB on disk.</br>

#### Medium: I chose this size

* `355M`: the "medium" model, 1.5GB on disk.</br>

#### Large and XL:  These two cannot be effectively fine-tuned on one GPU in colab </br>
(These Larger models have more knowledge, but take longer to finetune and longer to generate text.) 
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

 I specify which base model to use by changing `model_name` in the cells below.

Below, gpt-2 is downloaded from Google Cloud Storage and saved in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; I have to redownload it if I want to retrain it at a later time.


In [0]:
gpt2.download_gpt2(model_name="355M")

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount my personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [0]:
gpt2.mount_gdrive()

## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, I select *Files*. From there I upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
file_name = "songs.txt"

If the text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:
gpt2.copy_file_from_gdrive("songs.txt")

## Finetune GPT-2

My next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (If I want to have the finetuning run indefinitely, I can set `steps = -1`. I will not do that here.)

The model checkpoints are saved in `/checkpoint/run1`. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

(The training might time out after 4ish hours; I need to make sure I end training and save the results so I don't lose them!)

(**IMPORTANT NOTE:** To rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). I need to Rerun imports but I do not need to recopy files.

## optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [0]:
#Result Interp: 
#  (ie) [10 | 28.15] loss=3.44 avg=3.44
#  [NUMBER| TIME] loss=LOSS avg=AVG 
#1: NUMBER - Refers to the number of training step. Think of it as a counter that will increase by 1 after each run.
#2: TIME - elapsed since the start of training in seconds. 
#   We can use the first step as reference to determine how long does it take to run one step.
#3: LOSS and AVG -  Both of them refer to the cross-entropy (log loss) and the average loss.
#   We can use this to determine the performance of the model.
#   In theory, as training steps increases,the loss should decrease until it converges at certain value. 
#   The lower, the better.

sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=1000,
              restore_from='fresh',
              # latest
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

After the model is trained, copy the checkpoint folder to my Google Drive.

(To download it to my personal computer, copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file;It may be downloaded and uncompressed locally.)

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

Done with setup! Next is the **Generate Text From The Trained Model** section to generate text based on the retrained model.

## Load a Trained Model Checkpoint

Next, I copy the `.rar` checkpoint file from my Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** To rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). I will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

## Generate Text From The Trained Model

After training the model or loading a retrained model from checkpoint, I can now generate text. `generate` generates a single text from the loaded model.

In [0]:
gpt2.generate(sess, run_name='run1')

To create an API based on a model and to pass the generated text elsewhere, one can do `text = gpt2.generate(sess, return_as_list=True)[0]`

One can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if  adding an indicator when the text starts).

One can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, one can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

### Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
gpt2.generate(sess, 
              run_name='run4',
              prefix='SOURCE - ', 
              temperature=0.8,
              truncate='---',
              nsamples=10,
              batch_size=10,
              top_k=40)

# original  
# gpt2.generate(sess,
#               length=250,
#               temperature=0.7,
#               prefix="TITLE: ",
#               nsamples=5,
#               batch_size=5
#               )

For bulk generation, generate a large amount of text to a file and sort out the samples on a local computer. The next cell will generate a generated text file with a unique timestamp.

Rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

To generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [0]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

In [0]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

In [0]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

# **Results**
----------

There were pages of results, but I hand picked a few to illustrate the stages. 
## **Early runs:**

Repetition
```
TITLE: Tiki
Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki
Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki
Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki Tiki....
```

Nearly verbatim lyrics from The Aristocats, but adds a flourish here and there.
```
SOURCE -  The Aristocats
TITLE: The Aristocats
Which pet's address
Is the finest in Paris?
Which pet possess
The longest pedigree?
Which pets get to sleep
On velvet mats?
Which pets get a royal bath?
Which pets get to comb their
own voluptuouses?  <<< Totally made up word
```
gpt-2 drawing from a vast array of scientific knowledge to complete the Disnified lyrics, and odd capitalization
```
SOURCE -  Wreck-It Ralph
REMEMBER WHEN I SAID THAT WE'D GO SCIENCE FICTION? << Odd caps
Ralph is on the march again
Chasing a Scoville index
From Zero to 200
Fast-forward to today
And He's sold more than a billion books!
```
Combining Disney songs, but adding it's own flourish of gaming speak.
```
TITLE: I Just Can't Wait To Be King
Simba: I'm gonna be a mighty king
Just a mighty king
Just a mighty king
Saying: Be my guest
A Pint of the New Moon!
We surrender to your majesty!
Yes, we do.

```
Mary (Ice- )Poppins? Let it Go? 
```
TITLE: Supercalifragilisticexpialidocious  <<< Got it right! 
When an ice storm hits London,
And the ice doesn't last,
The ice queen will come to town.
She'll whip up a super-sized sleet,
That's super-super-super-super-cool. 
```

Outtakes: </br>
It just can't seem to make up it's mind! More repetition
```
TITLE: What's My Name?
What's my name?
What's my name?
What's my name?
What's my name?
```
Misspelling, but bonus points for use of "thy"? 
```
TITLE: Good Night
Good night
Don't delay
thy morning report  << Props for Shakesperian Lion King though
thy good brook
thy shore's free
all around lovely *blossombed* << b 
Tranquil as a child
Tranquil as a child
Good night
```


## **Mid runs:**
No. One. Else. Wrote. This. 
```
TITLE: Everything You Wanted (Only Elton John)  << ONLY EJ
'Cause everything we see, we must believe  << Starts with 'Cause. Who else? 
And the things we see, we must believe
This is the life we've been dreaming of
A life we can share
A simple life, as beautiful as it is wild
This is the life we want
A life we can share
A simple life, as beautiful as it is wild
Watch how we move, feel, and sing
Listen to the words that I'm singing
And feel the smile that comes to you
Just let me in, I'll show you how to be
Wild as all you want to be

```
Dear Abby...er gpt-2: Can I sum it up right here. (This is straight from reddit)
```
TITLE: Heartfelt   <<AWWWW 
tl;dr: I lost my way, and everyone looks like me  
<<<<<<<<<<<WE CAN TOTALLY SKIP THE REST>>>>>>>>>>>>>>
So I give myself a wide berth,
because I'm too trusting,
and I let the people I love tell me who they want to be.
They don't have to know who I am,
or where my heart is,
they can just see me do what I do.
I roll with the punches,
I sit back and wait for my turn,
when I get punched in the gut,
I get up and move on.
I see the world through,
and I see it differently,
I see the world through,
and I see there's nothing I can't do.
I roll with the punches,
I sit back and wait for my turn,
when I get punched in the gut,
I get up and move on.
I see the world through,
and I see it differently,
I see the world through.
And I see you,
here on my record,
I see you,
here on my conscience,
I see myself through,
and I see you
...everyday.

```
Is "Derek" being PA about "Who" vs "Whom" in this version of Jungle Book? ( Also from Reddit?)
```
TITLE: I Don't wanna be like you
Derek:  
Look you can't be friends with him
Me?
Both:
Friends with whom?
Derek:
With whom?
Me:
With whom?
Derek:
With whom?
Me:
Who?   << Yeah, I said it!  
Derek:
With whom? 
Me:
Whoever it may be. << Screw you, Derek. 
Derek and Me:
We want very much to be different.
We want to be perfect for each other.
Like Derek, Me and Derek.
Our relationship will not survive.
Me:
Will you just let me go...
Derek:
Look at him. I don't know what to do.
Me:
Leave him be. He'll learn.
Derek:
What to do?
Me:
What to do?
Derek:
Screw you.   <<< DID HE JUST???? 
Me:
I can't.
I can't do it.
I can't do it straight away.
I can't do it at all.
I can't do it in one go.
So please, somebody, tell me what to do.  << LEAVE HIM
Something's gotta give tonight.
It's my last one before I'm buried.
So if you're counting sheep, I'm counting goats.
And I'm counting on Derek to be my shepherd.
My sheep are restless, sad heads need a rest.
My goats are skinny yips, so I'm counting on Derek to be my shepherd.  <<<  OH NO!!  
My sheep are restless, sad heads need a rest.
And I'm counting on Derek to be my shepherd.
I'm counting on Derek to be my shepherd.  

(DONT DO IT, "Me"!) 
```


## **Late runs:**
Religiousity?    
```
TITLE: Theres A Party In Agrabah Part 2  << Part 2 
HEBREWS   
(Gasps with excitement) << !! 
JASMINE
You found him!
GARDNER
What now?
JASMINE
I'm giving him an ultimatum!
GARDNER
If he doesn't give me back the honey, I'll have to find a new way to eat!
JASMINE
(Neutral) 
```
Bare Nessesities: LifeTip DudeBro 2020 ( This isn't at all what I remembered Baloo imparting to Mowgli)
```
TITLE: The Bare Necessities
I've been around the block before
Stood in the back and avoided being hit
'Cause I've got something I learned from 'em.
The bare nessesities.
Like getting up in the morning and straightening up
'Cause those simple bare nessesities
Reduce stress and increase energy
And when you're stressed, it's great relief
To know that your energy levels will return to normal once you start bare nessing.
Once a week I'll double up as a lab rat
And I'll stick it out as long as I can
Because I've got something I learned from 'em.
The bare nessesities.
Like getting up in the morning and straightening up
'Cause those simple bare nessesities
Reduce stress and increase energy
And when you're stressed, it's great relief
To know that your energy levels will return to normal once you start bare nesses.
Once a week I'll double up as a lab rat
And I'll stick it out as long as I can
Because I've got something I learned from 'em.

```
Robin Williams gets an attribution for this amaglemation that sports some Hunchback refrerences and a very Tinkerbelle Movie "Foolharyette"! At Least there are no "Flubber" Refs!  
```
TITLE: Let Me Be Your Wings
Here I stand, alone and wide-eyed
In a burning building full of fire
Never dreaming why
All my dreams are like stars in the sky
Now here I stand, holding the sword in my fist
And the voice inside sings a song
Of fear and anger and love
And trust and kinship
And all it takes is one syllable
To say...
Let me be your wings  <LETMEBEYOURWINGS
Have you ever heard the wolf cry?
Or asked the smiling cow for milk
Or asked a dumbbells-n'-tenpenny how many pounds
You can carry?
Because if you haven't, you're a fool
and a foolhardyette
you should learn to be a fool
-- Robin Williams << 

```
Presented without comment, but I know what you are all thinking. 
```
TITLE: The American Adventure
Tell us what to do...

(<End of Song>)
```



## **Funny Titles**
```
TITLE: You Can Fly It's Just A Question Of When
TITLE: I'm Still Here (O Brother)
TITLE: I Spy (Rapunzel's Thing)
TITLE: I'm On Your Mind 
```


## **Bonus Round:  Titles from Prompt**

```
(Prefix = 'Title: Toss a Coin to Your Witcher')

Witcher Rap
TITLE: Toss a Coin to Your Witcher Hat
(bonus lyrics)
Do that for me!! Want a thrill???
Throw ya middle button. 
Hit mumble!
Don't let the middle card go unrewarded.


 (Now, That's some life advice if ever I heard it! Thanks gpt-2) 
```




## Which is Disney?  NO CHEATING! 
```
TITLE: My Love Will Find A Way
My love will find a way
To make me whole
Again
Again
Though he walked away
Our song was a prayer
We put the fear of God into you
And made you understand
What love is
My heart has wings and can fly
My love will find a way
To make me whole
Again
Again
```

```
TITLE: The Good Life
When I get to know him
For a time there I'll be safe and happy
Just let him be
Just let him be
Because I know he'll be there
When I get to know him better
For a time there I'll be safe and happy
Just let him be
Just let him be
Because I know he'll be there
```
## What do we think?  Should we recommend that Alan use this new trained model to help him? 

![Tiki Tiki](https://media.giphy.com/media/3ohze3TuWAtISYN6c8/giphy.gif) 




![Applause!](https://media.giphy.com/media/BYaQgGmxF4bgA/giphy.gif)

# **Thank You:**  
##Matt Vaughn </br>
##Max Woolf</br>
##Manuel Lara</br>
##Dylan Rossi  </br>
##Wenbin Xiao </br>
##Anselmo Jr Garza</br>
## Amy Koldeway, Sarah Cross and my classmates 





**Fun Fact:** **I trained with the same Voice teacher at [the same music school](https://millikin.edu/original-ariel-voice-launched-millikin) as Jodi Benson (aka Little Mermaid) which is why I'm so interested in Disney, Howard Ashman and Alan Menken**










